#  <center> Taller  de Aprendizaje Automático </center>
##  <center> Taller 4: Detección de Anomalías  </center>

#Introducción

En la siguiente actividad se trabajará en la detección de anomalías sobre redes de computadoras a partir de datos de tráfico. Para esto se utilizará una parte del conjunto [KDD Cup'99](https://scikit-learn.org/stable/datasets/real_world.html#kddcup99-dataset) pensada para evaluar métodos de detección de anomalías. 

Para los problemas de detección de anomalías generalmente no se cuenta con datos etiquetados para entrenar un detector. Por su definición las anomalías son eventos raros y por lo tanto poco frecuentes, lo que dificulta el etiquetado. Es por esto que este tipo de tareas generalmente son no supervisadas.

El enfoque más habitual para implementar soluciones para este tipo de problemas, es crear un modelo base a partir de un conjunto de datos "normales", es decir de los cuales se tenga cierta certeza de que todos fueron adquiridos en una situación normal. Luego en producción se detectarán como datos anómalos todos aquellos que no se ajusten a este modelo. Para saber el grado de ajuste de los datos se debe seleccionar un punto de operación, es decir, determinar cúando un dato se considera anómalo. En un ejemplo real, el cliente primero debería proporcionar un cantidad considerable de datos que representen el comportamiento normal de su sistema. Luego que se tiene el mejor modelo posible de estos datos, junto con el cliente, que es el que conoce su sistema, se debe determinar el punto de operación a partir del compromiso entre detectar la mayor cantidad de anomalías y obtener la menor cantidad de falsas alarmas posibles.

Para hacer investigación en la detección de anomalías, existen conjuntos de datos como el que se trabajará en esta actividad que si tienen etiquetas. Generalmente estas se obtienen probocando fallas y/o ataques intencionales a un sistema que se encuentra funcionando de manera normal. En esta actividad se separará el conjunto en dos partes. La primera con una gran proporción de datos etiquetados como normales, simulará ser el conjunto que el cliente nos proporciona para entrenar nuestro modelo. El otro conjunto tendrá datos etiquetados como normales o como anómalos, del cual se utilizará una mínima parte para definir el punto de operación y el resto simulará ser la puesta en producción.


## Objetivos


*   Abordar un problema de detección de anomalías, y ver las diferencias con un problema de clasificación convencional.
*   Trabajar con algoritmos de aprendizaje no supervisado.
*   Crear detectores compatibles con los *pipelines* de *scikit-learn*.


## Formas de trabajo

### Opción 1: Trabajar localmente

Descargar los datos en su máquina personal y trabajar en su propio ambiente de desarrollo.
 
*conda activate TAA-py38*             
*jupyter-notebook*    

Los paquetes faltantes se pueden instalar desde el notebook haciendo:     
*!pip install paquete_faltante*

### Opción 2:  Trabajar en *Colab*. 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/TAA-fing/TAA-2021/blob/main/taller2_criticas_cine.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Ejecutar en Google Colab</a>
  </td>
</table>

Se puede trabajar en Google Colab. Para ello es necesario contar con una cuenta de **google drive** y ejecutar un notebook almacenado en dicha cuenta. De lo contrario, no se conservarán los cambios realizados en la sesión. En caso de ya contar con una cuenta, se puede abrir el notebook y luego ir a *Archivo-->Guardar una copia en drive*.

# Datos
 
Los datos en total son 100655 y cuentan con 41 características. La columna *'labels'* indica si el dato es normal o, de no serlo, el tipo de anomalia.

*   Analizar y preprocesar los datos.
*   Cambiar las etiquetas de los datos de manera de tener sólo dos clases: normales y anómalos. ¿Cuál es la relación en cantidad entre ambas clases?

Manteniendo el orden en el que fueron descargados los datos.


*   Separar los primeros 90000 datos para el entrenamiento de los modelos. Estos deberían estar todos etiquetados como normales. 
*   Dividir el resto de los datos aleatoriamente de manera que el *10%* sea para la elección del punto de operación, y el restante *90%* para simular la puesta en producción. Asegurarse de que ambos tengan la misma proporción de anomalías.





In [ ]:
!pip install -U scikit-learn

In [ ]:
from sklearn.datasets import fetch_kddcup99
import pandas as pd
import numpy as np

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

KDDSA = fetch_kddcup99(subset='SA', as_frame=True, )

df = pd.DataFrame(data=KDDSA.frame.values, columns=KDDSA.frame.columns)
types = [float, str, str,str, float, float, str, float, float, float, float, str, float, float,float, float, float, float, float, float, str, str, 
         float, float, float, float,float, float, float, float,float, float, float, float, float, float, float,float, float, float, float, str]
columns = df.columns
for i in range(len(columns)):
  df[columns[i]] = df[columns[i]].astype(types[i])

print(df['labels'].value_counts())
print()


# Detección de anomalías

Si bien *scikit-learn* proporciona algunas herramientas para la detección de anomalías, esta actividad se centra en la utilización de algoritmos no supervisados generalmente pensados para otras tareas como: reducción de la dimensionalidad, y/o *clustering*. Especificamente se trabajará con: PCA, *K-Means*, y *Gaussian Mixture Model* (GMM).

## PCA



Para los datos de entrenamiento:
*   Aplicar PCA y graficar como varía el porcentaje de la varianza total en función de la cantidad de componentes principales (CPs). Se sugiere ver la sección *Choosing the Right Number of Dimensions* del capítulo 8 del libro.
*   Determinar la cantidad de CPs de manera de mantener el *99%* de la varianza de los datos.
*   Calcular y graficar el error de reconstrucción utilizando RMSE.

La forma más directa de hacer detección de anomalías utilizando PCA, es mediante el error de reconstrucción. Para esto primero se calculan los componentes principales a partir de los datos reservados para el modelado. Luego para cada dato a analizar se lo proyecta sobre estos componentes, y se calcula su reconstrucción. Debido a que los CPs fueron calculado sólo con datos normales, se espera que la reconstrucción de un dato anómalos tenga grandes errores. Es por esto que a partir del error de reconstrucción se pueda determinar si un dato es anómalo o no.

*   Implementar un detector tal como se describe arriba, utilizando RMSE para calcular el error. El mismo se debe definir como una clase de manera que sea compatible con los *pipelines* de *scikit-learn*. En la siguiente celda se muestra un *template* para crear la clase ([aquí](https://scikit-learn.org/stable/developers/develop.html) se pueden ver otros ejemplos).

*   Crear un *pipeline* que incluya el preprocesamiento y el detector implementado.
*   Entrenar el modelo de manera que mantenga el *99%* de la varianza. 
*   Proponga un punto de operación teniendo en cuenta que se quiere evitar un exceso de falsas alarmas. Para ello se recomienda graficar el compromiso entre *precision* y *recall* para distintos valores de *threshold* que definen el punto de operación. Ver la sección *Precision/Recall Trade-off* del capítulo 3 del libro.
*   Graficar los *scores* de los datos utilizados en el punto anterior, diferenciando con colores los datos normales de los anómalos. 


In [ ]:
from sklearn.base import BaseEstimator, OutlierMixin
from sklearn.utils.validation import check_array, check_is_fitted
from sklearn.decomposition import PCA

class AD_PCA(BaseEstimator, OutlierMixin):

  def __init__(self, n_components=None):
    self.n_comp = n_components

  def fit(self, X, y=None):
    self.X = X
    self.y = y 
    self.PCA_ = PCA(n_components=self.n_comp)
    self.PCA_.fit(X)
    return self

  def score(self, X, y=None):
    X = check_array(X)
    check_is_fitted(self, ['X', 'y'])

    # Agregar código---

    #------------------
    return score

## K-Means

*   Para los datos de entrenamiento proponer una forma de hallar el valor de *K* óptimo. Se sugiere ver la sección *Finding the optimal number of clusters* del capítulo 9. 
*   Siguiendo en la misma linea, ¿cómo implementaría un detector de anomalías utilizando *K-Means*?.
*   Crear una clase y un *pipeline* para la solución propuesta.
*   Eligir un punto de operación siguiendo un procedimiento similar al de la parte anterior.
*   Graficar el *score* para los datos utilizados en el punto anterior. 


In [ ]:
from sklearn.cluster import KMeans

## Gaussian Mixtures Models

Siguiendo el ejemplo de la sección *Anomaly Detection Using Gaussian Mixture* en el capítulo 9 del libro. 

*   Implementar un detector que calcule el valor de los *scores*, y que además determine las predicciones a partir de un *threshold* calculado de manera similar al ejemplo. En la siguiente celda se proporciona un *template* para la implementación del detector.
*   ¿Cuántos componentes está utilizando para la mezlca? ¿por qué?.

*   Obtener diferentes predicciones para los datos del punto de operación,variando el parámetro *percen*. Discutir sobre los resultados.

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
class AD_GMM(BaseEstimator, OutlierMixin):

  def __init__(self, n_components=1, percen=0):
    self.n_comp = n_components
    self.percen = percen

  def fit(self, X, y=None):
    self.classes_ = [1, 0]
    self.X = X
    self.y = y 
    self.GMM_ = GaussianMixture(n_components=self.n_comp)
    self.GMM_.fit(X)

    #threshold---

    #------------
    return self

  def score(self, X, y=None):
    X = check_array(X)
    check_is_fitted(self, ['X', 'y'])

    # Agregar código---

    #------------------
    return score

  def predict(self, X):
    X = check_array(X)
    check_is_fitted(self, ['X', 'y'])

    # Agregar código---

    #------------------
    return pred 

# Opcional

*   Aplicar a los datos del problema alguno de los detectores de *sikit-learn* como: One-Class SVM, Isolation Forest
*   Comparar con los detectores anteriores.

